In [ ]:
import pandas as pd
import re
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.svm import SVC
import numpy as np

#Data Processing Packages
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection

#Data Visualization Packages
import matplotlib.pyplot as plt
import seaborn as sns

#Machine Learning Packages
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn import model_selection
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

# %matplotlib tk

def plot_corr(df):
    """
    Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot

    Displays:
        matrix of correlation between columns.  Blue-cyan-yellow-red-darkred => less to more correlated
                                                0 ------------------>  1
                                                Expect a darkred line running from top left to bottom right
    """
    sns.heatmap(df.corr(), annot=True)

def plot_bar(target, df, exclude=tuple()):
    total_plots = df.shape[1]
    ncols=3
    nrows = math.ceil(total_plots/ncols)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(nrows*ncols, nrows*ncols))
    i=0
    j=0
    for col_name in df.columns: 
#         print(col_name)
#         print("{0}-{1}".format(i,j))
        
        if (col_name == target) or col_name in exclude:
            continue
        try:
            sns.barplot(x=target, y=col_name, data=df, ax=ax[i,j])
        except Exception as e:
            print("WARNING unable to plot for {} against {}".format(target, col_name))
        j += 1
        if j == (ncols):
            i += 1
            j = 0
# Function to calculate no. of null values with percentage in the dataframe
def null_values(DataFrame_Name):
    
    sum_null = DataFrame_Name.isnull().sum()
    total_count = DataFrame_Name.isnull().count()
    percent_nullvalues = sum_null/total_count * 100
    df_null = pd.DataFrame()
    df_null['Total_values'] = total_count
    df_null['Null_Count'] = sum_null
    df_null['Percent'] = percent_nullvalues
    df_null = df_null.sort_values(by='Null_Count',ascending = False)

    return(df_null)

# Function to calculate no. of null values with percentage in the dataframe
def specific_values(DataFrame_Name, value=0):
    
    sum_by_value = (DataFrame_Name ==value).sum()
    total_count = (DataFrame_Name ==value).count()
    percent_byvalues = sum_by_value/total_count * 100
    df_null = pd.DataFrame()
    df_null['Total_values'] = total_count
    df_null['value_Count'] = sum_by_value
    df_null['Percent'] = percent_byvalues
    df_null = df_null.sort_values(by='value_Count',ascending = False)

    return(df_null)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

df = pd.read_csv("../mine/train2.csv", low_memory=False)
print('Setup Completed')
df.columns
# Any results you write to the current directory are saved as output.

In [ ]:
# null_values(df)
df['index_col'] = df.index

truncated_df =  df[['BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS',
       'BEDRM',  'YR_RMDL', 'EYB', 'STORIES',  'PRICE',
       'QUALIFIED',  'GBA',  'STYLE', 'STRUCT', 'GRADE',
       'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 
        'LANDAREA',   'index_col','STATE','SQUARE','QUADRANT'
        
       
       ]]
truncated_df['YR_RMDL'] = truncated_df['YR_RMDL'].fillna(truncated_df['EYB'])

del truncated_df['EYB']


In [ ]:
# IMPUTATIONS
# replace 50% null values of YR_RMDL with AYB


truncated_df["STATE"] = truncated_df["STATE"].fillna(truncated_df["STATE"].mode()[0])
# truncated_df["CITY"] = truncated_df["CITY"].fillna(truncated_df["CITY"].mode()[0])
truncated_df["STRUCT"] = truncated_df["STRUCT"].fillna(truncated_df["STRUCT"].mode()[0])
truncated_df["STYLE"] = truncated_df["STYLE"].fillna(truncated_df["STYLE"].mode()[0])
truncated_df["INTWALL"] = truncated_df["INTWALL"].fillna(truncated_df["INTWALL"].mode()[0])
truncated_df["EXTWALL"] = truncated_df["EXTWALL"].fillna(truncated_df["EXTWALL"].mode()[0])
truncated_df["GRADE"] = truncated_df["GRADE"].fillna(truncated_df["GRADE"].mode()[0])
# truncated_df["ZIPCODE"] = truncated_df["ZIPCODE"].fillna(truncated_df["ZIPCODE"].mode()[0])
# truncated_df.dropna(subset = ['PRICE'],inplace = True)
truncated_df["PRICE"] = truncated_df["PRICE"].fillna(truncated_df["PRICE"].mean())
# truncated_df["SQUARE"][truncated_df["SQUARE"]=="PAR "] = truncated_df["SQUARE"][truncated_df["SQUARE"]!="PAR "].mean()

truncated_df["STORIES"] = truncated_df["STORIES"].fillna(truncated_df["STORIES"].mode()[0])
truncated_df["KITCHENS"] = truncated_df["KITCHENS"].fillna(truncated_df["KITCHENS"].mode()[0])
truncated_df["NUM_UNITS"] = truncated_df["NUM_UNITS"].fillna(truncated_df["NUM_UNITS"].mode()[0])
truncated_df["GBA"] = truncated_df["GBA"].fillna(truncated_df["GBA"].mode()[0])
truncated_df["SQUARE"] = truncated_df["SQUARE"].fillna(truncated_df["SQUARE"].mode()[0])
truncated_df["ROOMS"] = truncated_df["ROOMS"].fillna(truncated_df["ROOMS"].mode()[0])
truncated_df["BATHRM"] = truncated_df["BATHRM"].fillna(truncated_df["BATHRM"].mode()[0])
truncated_df["HF_BATHRM"] = truncated_df["HF_BATHRM"].fillna(truncated_df["HF_BATHRM"].mode()[0])
truncated_df["LANDAREA"] = truncated_df["LANDAREA"].fillna(truncated_df["LANDAREA"].mode()[0])


null_values(truncated_df)
# truncated_df[truncated_df["YR_RMDL"].isnull()==1]


In [ ]:
pd.set_option('display.max_columns', 500)
null_values(truncated_df)
df.head(10)
# truncated_df["SQUARE"].value_counts()

In [ ]:
truncated_df.shape
from sklearn.model_selection import train_test_split
truncated_df["HAC"] =  truncated_df["AC"]
truncated_df["BBRM"] = truncated_df["BATHRM"] +truncated_df["HF_BATHRM"]+truncated_df["BEDRM"]  + truncated_df["ROOMS"] 

feature_col_names = ['HAC',   'BBRM','YR_RMDL', 'STORIES',  'STYLE',  'GRADE', 'EXTWALL', 'LANDAREA', 'NUM_UNITS',\
                    'INTWALL', 'STRUCT', 'GBA', 'KITCHENS','QUALIFIED','ROOF','CNDTN','HEAT','STATE','QUADRANT']
#'GBA', 'STYLE',  'GRADE', 'CNDTN',  
       # 'KITCHENS', 'LANDAREA', 'index_col'
predicted_class_names = ['PRICE']


X = truncated_df[feature_col_names]     # predictor feature columns (8 X m)
Y = truncated_df[predicted_class_names] # predicted class (1=true, 0=false) column (1 X m)
Y["PRICE"] = Y["PRICE"].astype(int)
split_test_size = 0.30
X['BBRM'] = pd.qcut(X['BBRM'], 4, duplicates="drop")
# X['BEDRM'] = pd.qcut(X['BEDRM'], 4, duplicates="drop")
X['NUM_UNITS'] =pd.qcut(X['NUM_UNITS'], 4, duplicates="drop")


X['STORIES'] = pd.qcut(X['STORIES'], 4, duplicates="drop")
# X['SALE_NUM'] = pd.qcut(X['SALE_NUM'], 4, duplicates="drop")
X['YR_RMDL'] = pd.qcut(X['YR_RMDL'], 4, duplicates="drop")
X['GBA'] = pd.qcut(X['GBA'], 4, duplicates="drop")
X['KITCHENS'] = pd.qcut(X['KITCHENS'], 4, duplicates="drop")
X['LANDAREA'] = pd.qcut(X['LANDAREA'], 4, duplicates="drop")

# X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=split_test_size, random_state=42) 
cols=['HAC',   'BBRM','YR_RMDL', 'STORIES','STYLE',  'GRADE', 'EXTWALL', 'LANDAREA','NUM_UNITS',\
                    'INTWALL', 'STRUCT', 'GBA', 'KITCHENS', 'QUALIFIED','ROOF','CNDTN','HEAT','STATE','QUADRANT'
       
         ]
X = pd.get_dummies(X, columns=cols, prefix=cols)
X_train=X[:2000]
X_test=X[2300:3000]
Y_train=Y[:2000]
Y_test=Y[2300:3000]
# del X_train['index_col']
# del X_test['index_col']


# X_train = pd.get_dummies(X_train, columns=['HAC',   'BBRM','YR_RMDL', 'STORIES','STYLE',  'GRADE',
       
#          ], \
#                        prefix=['HAC',   'BBRM','YR_RMDL', 'STORIES','STYLE',  'GRADE',
#         ])
# X_test = pd.get_dummies(X_test, columns=['HAC',   'BBRM','YR_RMDL', 'STORIES',  'STYLE',  'GRADE',
       
#         ], \
#                        prefix=['HAC',   'BBRM','YR_RMDL', 'STORIES', 'STYLE',  'GRADE',
       
#          ])
X_train.shape

In [ ]:
# Modeling step Test differents algorithms 
cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=.3, train_size=.7, random_state=42)

classifiers = [
#     SVC(random_state=42, probability=True),
#     DecisionTreeClassifier(random_state=42),
#     AdaBoostClassifier(DecisionTreeClassifier(random_state=42),random_state=42,learning_rate=0.1),
#     RandomForestClassifier(random_state=42),
#     ExtraTreesClassifier(random_state=42),
#     GradientBoostingClassifier(random_state=42),
#     MLPClassifier(random_state=42),
#     KNeighborsClassifier(),
    LogisticRegression(random_state=42),
#     LinearDiscriminantAnalysis(),
#     XGBClassifier()
]

cv_train_mean = []
cv_test_mean = []
cv_score_time = []
cv_fit_time = []
cv_name = []
predictions = []

for classifier in classifiers :
    cv_results = model_selection.cross_validate(classifier, X_train, Y_train, cv=cv_split, return_train_score=True)
    cv_train_mean.append(cv_results['train_score'].mean())
    cv_test_mean.append(cv_results['test_score'].mean())
    cv_score_time.append(cv_results['score_time'].mean())
    cv_fit_time.append(cv_results['fit_time'].mean())
    cv_name.append(str(classifier.__class__.__name__))
    classifier.fit(X_train, Y_train)
    predictions.append(classifier.predict(X_test))
    

performance_df = pd.DataFrame({"Algorithm":cv_name, "Train Score":cv_train_mean, "Test Score":cv_test_mean, 'Score Time':cv_score_time, 'Fit Time':cv_fit_time})
performance_df